# Homework 4
## Due October 22

## 1 (20 pts) Backpropagation
While the formulas for backpropagation that we've dealt with are generally for fairly large graphs, it's instructive to develop a little bit of intuition by performing it on very small graphs.  Consider the following graph:
<img src=graph.png>
where $x$ is an input, nodes with a $\sigma$ apply the logistic function, nodes with an $I$ apply the identity function (aka don't really do anything), and $L$ is the squared error between a prediction and an observation $y$.  Written out algebraically, this graph says
\begin{align}
a^{(1)}& = x w_1^{(1)} \\
z^{(1)}& = \sigma(a_1) \\
[a^{(2)}_1,a^{(2)}_2]& = z^{(1)} \times [w_1^{(2)}, w_2^{(2)}] \\
[z^{(2)}_1,z^{(2)}_2]& = [\sigma(a^{(2)}_1),\sigma(a^{(2)}_2)] \\
a^{(3)}& = z^{(2)}_1 w_1^{(3)} + z^{(2)}_2 w_2^{(3)} \\
z^{(3)}& = a^{(3)} \\
L &= \frac{1}{2}(z^{(3)} - y)^2 \\
\end{align}
The chain rule can be used to take derivatives of $L$ with respect to the various weights, even far back in the chain.  For example, we could use the chain rule to take the derivative of $L$ with respect to $w^{(3)}_1$ as follows:
$$
\frac{\partial L}{w_1^{(3)}} = \frac{\partial L}{\partial z^{(3)}} \frac{\partial z^{(3)}}{\partial a^{(3)}} \frac{\partial a^{(3)}}{\partial w^{(3)}_1} = (z^{(3)} - y) \times 1 \times z^{(2)}_1
$$
What we've done here is to trace the path from $L$ to the parameter, taking the derivative between the outputs and inputs of each node as we work back through the graph, then evaluated each resulting derivative!

### 1A 
**Derive and evaluate an expression for $$\frac{\partial L}{\partial w^{(2)}_2}.$$**  Compute this derivative for the feature $x$, data $y$, and parameter values $w_j^{(l)}$ given in the following code block.  Compare your results to the [finite difference approximation](https://en.wikipedia.org/wiki/Finite_difference_method) of this derivative computed below. 

In [ ]:
import numpy as np

x = 1.0      # Single feature
w_11 = 0.5   # Parameters
w_12 = 0.7
w_22 = -0.3
w_13 = 0.1
w_23 = -0.8
y = 0.5      # Single data point

# Sigmoid function
def sigmoid(a):
    return 1./(1+np.exp(-a))

# Forward pass
def forward(x):
    a_1 = x*w_11
    z_1 = sigmoid(a_1)
    a_12,a_22 = z_1*w_12,z_1*w_22
    z_12,z_22 = sigmoid(a_12),sigmoid(a_22)
    a_3 = z_12 * w_13 + z_22 * w_23
    z_3 = a_3
    L = 0.5*(z_3 - y)**2
    return L

# Compute the forward pass
L_0 = forward(x)

# Very slightly change the value of w_22
w_22 += 1e-5

# Compute the forward pass again
L_1 = forward(x)

# Compute Delta L / Delta w_22 as an approximation for the derivative
dLdw22_fd = (L_1 - L_0)/1e-5

# Reset w_22 to its previous value
w_22 -= 1e-5
print('The finite difference approximation to dL/dw_2^2 is: ',dLdw22_fd)

#! Your function to evaluate the derivative symbolically goes here.

### 1B (FOR GRAD STUDENTS ONLY)
**Why shouldn't we just use the finite difference method like the one shown above to compute gradients?  Why is backpropagation useful?** 

### 1C (FOR GRAD STUDENTS ONLY) 
**Use the chain rule to determine an expression for**
$$
\frac{\partial L}{\partial w_1^{(1)}}
$$
This one is a little bit harder because there are *two paths* from $w_1^{(1)}$ to $L$ in the graph (unlike in Part 1A)!  There are many ways to deal with this if you have the calculus background to do it, but perhaps the most intuitive way (and what automatic differentiation systems do) is to evaluate the chain rule for both paths and *sum them*.   

## 2 Facial Recognition

The same type of neural network that we've been using to classify digits can just as easily be used to classify other image datasets.  A particularly fun and easy dataset to work with is called the [Labeled Faces in the Wild](http://vis-www.cs.umass.edu/lfw/) dataset, which contains a large database of cropped grayscale images with associated labels (i.e. names).  We can download it using sklearn.datasets as follows:

In [ ]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

# Fetch LFW dataset, ensuring that we have at least 50 images per class (i.e. per person)
lfw_people = fetch_lfw_people(min_faces_per_person=50, resize=0.4)

# Extract number of data points, and the height and width of the images for later reshaping
m, h, w = lfw_people.images.shape
n = h*w

# Extract number of classes
N = len(lfw_people.target_names)

# Split the training and test set
X,X_test,y,y_test = train_test_split(lfw_people.data,lfw_people.target)
X/=255.
X_test/=255.

## 2A Facial Recognition System (40 pts)

**Using the MNIST classifier we developed in pytorch for class as a basis for development, create a new classifier that inputs faces and outputs names.**  
- 1) you'll have to change some things, in particular the number of input features, the number of hidden layer nodes, and also the number of classes!  
- 2) To begin with, use only 8 nodes in your hidden layer.
- 3) Adjust the learning rate, number of epochs, and batch size so that you're confident the network is converged, based either on the cost function or on the test set accuracy

**Train your classifier and report your test set accuracy!**

## 2B Model Selection (20pts)
We began with a relatively simple model that had only 8 hidden nodes.  If we increase that number do we get better predictive accuracy on the test set?  **Re-run your model using 16 hidden layer nodes.  Is the increase in model complexity (and computing effort) justified?  Why or why not?**  If you answered yes to the previous question, double the number of nodes again, and answer the same questions.  **What is the maximum number of hidden layer nodes that is justified for this task?**

## 2C Feature selection visualization (20 pts)
As in our experiments with MNIST, we're interested in determining exactly what features this neural network is extracting from the raw input data.  **Extract the weight matrix between the input and hidden layer, then reshape and visualize a few of them as images** (note the image width and height parameters defined when we loaded the data).  **Comment on the types of features that this neural network is paying attention to.**

- Note 1) We can extract parameters from our Model class (and transform them into numpy arrays) with the command:

In [ ]:
params = [p.cpu().detach().numpy() for p in model.l1.parameters()]

- Note 2) Pytorch defines the linear transformation between activation layers using matrices that are transposed relative to how we've seen it so far, i.e. as  
$$
A = X W^T + B^T
$$
This means that you'll want to extract and reshape *rows* rather than *columns* of this weight matrix for visualization as an image.
